<h3>Dependency Installation</h3>

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE

<h3>Data Cleaning</h3>

In [2]:
applications = pd.read_csv('Application Data.csv')
applications.dropna(subset=['Username'], inplace=True)

In [3]:
def ConvertFound(found: str) -> str:
    if found == 'Shouts':
        return 'Shout'
    elif found == 'Friend' or found == 'Other':
        return 'Other'
    elif found == 'Forums' or found == 'Wynn Discord':
        return 'External'
    elif found == 'In-game Party' or found == 'Map':
        return 'In-Game'
    else:
        return np.NaN

In [4]:
def ConvertActivity(activity: str) -> str:
    if activity == 'No life' or activity == 'Daily':
        return activity
    elif activity == 'Sometimes' or activity == 'Weekly':
        return 'Other'
    else:
        return np.NaN

In [5]:
def ConvertPreviousGuild(previousGuild: str) -> bool:
    if previousGuild == 'Rando' or previousGuild == 'Allies' or previousGuild == 'Attacker' or previousGuild == 'Adonis':
        return True
    elif np.isnan(previousGuild):
        return False
    else:
        return np.NaN

In [6]:
def GuildGrindContributions() -> pd.Series:
    guildGrind = ['Warring', 'Grinding', 'Guild Objective']
    cont_1 = applications['Contribute 1'].apply(lambda x: 1 if x in guildGrind else 0)
    cont_2 = applications['Contribute 2'].apply(lambda x: 1 if x in guildGrind else 0)
    cont_3 = applications['Contribute 3'].apply(lambda x: 1 if x in guildGrind else 0)
    cont_4 = applications['Contribute 4'].apply(lambda x: 1 if x in guildGrind else 0)
    
    return (cont_1 + cont_2 + cont_3 + cont_4).astype(bool)

In [7]:
def CommunityContributions() -> pd.Series:
    community = ['Helping Others', 'Chatting']
    cont_1 = applications['Contribute 1'].apply(lambda x: 1 if x in community else 0)
    cont_2 = applications['Contribute 2'].apply(lambda x: 1 if x in community else 0)
    cont_3 = applications['Contribute 3'].apply(lambda x: 1 if x in community else 0)
    cont_4 = applications['Contribute 4'].apply(lambda x: 1 if x in community else 0)

    return (cont_1 + cont_2 + cont_3 + cont_4).astype(bool)

In [8]:
def CountActivities() -> pd.Series:
    activity_1 = applications['Enjoy 1'].isna().astype(int)
    activity_2 = applications['Enjoy 2'].isna().astype(int)
    activity_3 = applications['Enjoy 3'].isna().astype(int)

    return activity_1 + activity_2 + activity_3

In [9]:
def CountHobbies() -> pd.Series:
    hobby_1 = applications['Hobbies 1'].isna().astype(int)
    hobby_2 = applications['Hobbies 2'].isna().astype(int)
    hobby_3 = applications['Hobbies 3'].isna().astype(int)
    hobby_4 = applications['Hobbies 4'].isna().astype(int)

    return hobby_1 + hobby_2 + hobby_3 + hobby_4

In [10]:
applications_ = pd.DataFrame()

applications_['Age'] = applications['Age'].fillna(round(applications['Age'].mean()))
applications_['Level'] = applications['Level']
applications_['Activity'] = applications['Activity'].apply(ConvertActivity).map({'Other': 0, 'Daily': 1, 'No life': 2})
applications_['Found'] = applications['Found'].apply(ConvertFound).map({'Other': 0, 'In-Game': 1, 'Shout': 2, 'External': 3})
applications_['Past Guild'] = applications['Prev guilds'].apply(ConvertPreviousGuild).map({False:0, True:1})

applications_['Guild Grind'] = GuildGrindContributions().map({False:0, True:1})
applications_['Community'] = CommunityContributions().map({False:0, True:1})

applications_['Num Enjoy'] = CountActivities()
applications_['Num Hobbies'] = CountHobbies()

applications_['Successful'] = applications['Successful'].map({'Yes': True, 'No': False, 'Trouble': False, 'Unknown': np.NaN}).map({False:0, True:1})

applications_.dropna(inplace=True)

In [11]:
applications_.sample(n=20)

,Age,Level,Activity,Found,Past Guild,Guild Grind,Community,Num Enjoy,Num Hobbies,Successful
43,19.0,105.0,1,0,1,1,0,1,3,0
33,18.0,43.0,1,2,0,0,0,1,3,0
327,18.0,103.0,2,2,1,1,1,1,2,1
141,18.0,71.0,1,0,1,1,0,2,2,0
38,20.0,106.0,1,1,1,1,0,1,2,1
180,18.0,56.0,0,1,0,1,0,1,2,0
355,18.0,105.0,0,1,1,1,1,2,1,0
424,18.0,106.0,1,0,1,1,0,1,3,0
456,18.0,93.0,1,2,1,0,0,2,3,0
238,15.0,78.0,0,2,1,0,0,2,2,0


<h3> Model Creation </h3>

In [12]:
data = applications_[['Age', 'Level', 'Activity', 'Found', 'Past Guild', 'Guild Grind', 'Community', 'Num Enjoy', 'Num Hobbies']]
labels = applications_['Successful']
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.35)

train_data_rs, train_labels_rs = SMOTE().fit_resample(train_data, train_labels)

In [17]:
model_rs = make_pipeline(StandardScaler(), SVC(class_weight='balanced', cache_size=500))
model_rs.fit(train_data_rs, train_labels_rs)

tn, fp, fn, tp = confusion_matrix(y_pred=model_rs.predict(test_data), y_true=test_labels).ravel()
print(tn, fp, fn, tp)

print(model_rs.score(test_data, test_labels))

122 36 9 8
0.7428571428571429


In [18]:
model = make_pipeline(StandardScaler(), SVC(class_weight='balanced', cache_size=500))
model.fit(train_data, train_labels)

tn, fp, fn, tp = confusion_matrix(y_pred=model.predict(test_data), y_true=test_labels).ravel()
print(tn, fp, fn, tp)

print(model.score(test_data, test_labels))

104 54 7 10
0.6514285714285715
